In [1]:
!pip install --upgrade openai

     ---------------------------------------- 0.0/107.3 kB ? eta -:--:--
     --- ------------------------------------ 10.2/107.3 kB ? eta -:--:--
     ---------- -------------------------- 30.7/107.3 kB 660.6 kB/s eta 0:00:01
     -------------- ---------------------- 41.0/107.3 kB 281.8 kB/s eta 0:00:01
     ---------------------------- -------- 81.9/107.3 kB 512.0 kB/s eta 0:00:01
     ------------------------------------ 107.3/107.3 kB 567.7 kB/s eta 0:00:00
   ---------------------------------------- 0.0/320.7 kB ? eta -:--:--
   ---------- ----------------------------- 81.9/320.7 kB 1.5 MB/s eta 0:00:01
   -------------- ------------------------- 112.6/320.7 kB 2.2 MB/s eta 0:00:01
   ----------------- ---------------------- 143.4/320.7 kB 1.1 MB/s eta 0:00:01
   ------------------------------ --------- 245.8/320.7 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 320.7/320.7 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/85.6 kB ? eta -

In [2]:
!pip install tiktoken

     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     --------- ------------------------------ 10.2/42.0 kB ? eta -:--:--
     ------------------ ------------------- 20.5/42.0 kB 217.9 kB/s eta 0:00:01
     --------------------------- ---------- 30.7/42.0 kB 187.9 kB/s eta 0:00:01
     -------------------------------------- 42.0/42.0 kB 203.4 kB/s eta 0:00:00
   ---------------------------------------- 0.0/799.0 kB ? eta -:--:--
    --------------------------------------- 10.2/799.0 kB ? eta -:--:--
   - ------------------------------------- 41.0/799.0 kB 653.6 kB/s eta 0:00:02
   -- ------------------------------------ 61.4/799.0 kB 465.5 kB/s eta 0:00:02
   ----- -------------------------------- 112.6/799.0 kB 652.2 kB/s eta 0:00:02
   -------- ----------------------------- 174.1/799.0 kB 871.5 kB/s eta 0:00:01
   --------- ---------------------------- 204.8/799.0 kB 778.2 kB/s eta 0:00:01


In [4]:
import pandas as pd
import tiktoken

from openai import OpenAI
client = OpenAI(api_key="sk-")

# set text-Embedding modal default to text-embedding-ada-002
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [5]:

# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8196

In [7]:
input_datapath = "fine_food_reviews_1k.csv"  # to save space, we provide a pre-filtered dataset
df = pd.read_csv(input_datapath, index_col=0)
df = df[["Time", "ProductId", "UserId", "Score", "Summary", "Text"]]
df = df.dropna()
df["combined"] = (
    "Title: " + df.Summary.str.strip() + "; Content: " + df.Text.str.strip()
)
df.head(2)

,Time,ProductId,UserId,Score,Summary,Text,combined
0,1351123200,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...
1,1351123200,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",Title: Arrived in pieces; Content: Not pleased...


In [8]:

# subsample to 1k most recent reviews and remove samples that are too long
top_n = 1000
df = df.sort_values("Time").tail(top_n * 2)  # first cut to first 2k entries, assuming less than half will be filtered out
df.drop("Time", axis=1, inplace=True)

encoding = tiktoken.get_encoding(embedding_encoding)

# omit reviews that are too long to embed, make sure the there will be no runtime error
df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))
df = df[df.n_tokens <= max_tokens].tail(top_n)
len(df)

1000

In [11]:
# Ensure you have your API key set in your environment per the README: https://github.com/openai/openai-python#usage

# This may take a few minutes
df["embedding"] = df.combined.apply(lambda x: get_embedding(x, model=embedding_model))
df.to_csv("fine_food_reviews_with_embeddings_1k.csv")


APITimeoutError: Request timed out.